In [ ]:
from matplotlib import pyplot,cm
from math import pi
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
import numba
from numba import jit

In [ ]:
ni=51
nj=51
dx=2*pi/(ni-1)
dy=2*pi/(nj-1)
x=numpy.linspace(-dx/2,2*pi+dx/2,ni+1)
y=numpy.linspace(-dy/2,2*pi+dy/2,nj+1)

X,Y=numpy.meshgrid(x,y)

u0=-numpy.cos(X)*numpy.sin(Y)
v0=numpy.sin(X)*numpy.cos(Y)
p0=numpy.cos(2*X)+numpy.cos(2*Y)

In [ ]:
omega0=numpy.zeros((nj+1,ni+1))
for i in range(1,ni):
    for j in range(1,nj):
        omega0[j,i]=(v0[j,i+1]-v0[j,i])/dx-(u0[j+1,i]-u0[j,i])/dy

In [ ]:
def Fc1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=-((u[1:-1,1:-1]+u[1:-1,2:])**2-(u[1:-1,1:-1]+u[:-2])**2)/(4*dx)-\
                ((u[1:-1,1:-1]+u[2:,1:-1])*(v[1:-1,2,:]+v[1:-1,1:-1])-(u[1:-1,1:-1]+u[:-2,1:-1])*(v[:-2,2:]+v[:-2,1:-1]))/(4*dy)
        
    F[:,0]=F[:,1]
    F[:,-1]=F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [ ]:
def Fv1(u,v,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=(u[1:-1,2:]-2*u[1:-1,1:-1]+u[1:-1,:-2])/dx**2+(u[2:,1:-1]-2*u[1:-1,1:-1]+u[:-2,1:-1])/dy**2
    
    F[:,0]=F[:,1]
    F[:,-1]=F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [ ]:
def Fp1(p,dx,dy):
    F=numpy.zeros(numpy.shape(u))
    F[1:-1,1:-1]=-(p[1:-1,2,:]-p[1:-1,1:-1])/dx
    F[:,0]=F[:,1]
    F[:,-1]=F[:,-2]
    F[0,:]=-F[1,:]
    F[-1,:]=-F[-2,:]
    return F

In [ ]:
def Fc2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=-((u[2:,1:-1]+u[1:-1,1:-1])*(v[1:-1,1:-1]+v[1:-1,2:])-(u[2:,:-2]+u[1:-1,:-2])*(v[1:-1,1:-1]+v[1:-1,:-2]))/(4*dx)-\
                 ((v[1:-1,1:-1]+[v[2:,1:-1]])**2-(v[1:-1,1:-1]+v[:-2,1:-1])**2)/(4*dy)
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=F[1,:]
    F[-1,:]=F[-2,:]
    return F

In [ ]:
def Fv2(u,v,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=(v[1:-1,2:]-2*v[1:-1,1:-1]+v[1:-1,:-2])/dx**2+(v[2:,1:-1]-2*v[1:-1,1:-1]+v[:-2,1:-1])/dy**2
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=F[1,:]
    F[-1,:]=F[-2,:]
    return F

In [ ]:
def Fp2(p,dx,dy):
    F=numpy.zeros((numpy.shape(u)))
    F[1:-1,1:-1]=-(p[2:,1:-1]-p[:-2,1:-1])/dy
        
    F[:,0]=-F[:,1]
    F[:,-1]=-F[:,-2]
    F[0,:]=F[1,:]
    F[-1,:]=F[-2,:]
    return F

In [ ]:
def poisson(f,res_target,omega):
    p=numpy.zeros(numpy.shape(f))
    res_max=1e3
    while res_max>res_target:
        
        for j in range(1,nj):
            for i in range(1,ni):
                p[j,i]=omega/(2*(dx**2+dy**2))*((p[j,i-1]+p[j,i+1])*dy**2+\
                       (p[j-1,i]+p[j+1,i])*dx**2-f[j,i]*dx**2*dy**2)+\
                       (1-omega)*u[j,i]
                    
        p[0,:]=p[1,:]
        p[-1,:]=p[-2,:]
        p[:,0]=p[:,1]
        p[:,-1]=p[:,-2]
        
        for j in range(1,nj):
            for i in range(1,ni):
                res[j,i]=numpy.abs(f[j,i]-(p[j,i-1]-2*p[j,i]+p[j,i+1])/dx**2-\
                                  (p[j-1,i]-2*p[j,i]+p[j+1,i])/dy**2)
        res_max=numpy.max(res)
        
    return p

In [ ]:
def grad(v,dx,dy):
    g=numpy.zeros(numpy.shape(v))
    g[1:-1,1:-1]=(v[:,2:]-v[:,:-2])/2/dx+(v[2:,:]-v[:-2,:])/2/dy
    return g

In [ ]:
dt=0.05
T=10
nt=int(T/dt)
t=0
u=u0.copy()
v=v0.copy()
p=p0.copy()
for t in range(nt):
    
    Gu=Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
    vu=u+dt*Gu/3
    p1=poisson(3./dt*(grad(v1)))
    Fp_1=Fp1(p,dx,dy)+Fp2(p,dx,dy)
    u=v_1+dt/3*Fp_1
    
    G_2=-5./9*G_1+Fc1(u,v,dx,dy)+Fv1(u,v,dx,dy)
    vi=

In [ ]:
pyplot.figure(figsize=(8,6))
#pyplot.contour(X,Y,2*numpy.cos(x)*numpy.cos(Y),10)#,cmap=cm.viridis)
pyplot.contour(X,Y,omega0,10)
pyplot.colorbar();
pyplot.quiver(X[::3,::3],Y[::3,::3],u0[::3,::3],v0[::3,::3],scale=13)
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')